In [14]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import zipfile
import rasterio as rio
from pyproj import Proj, transform

import warnings
warnings.filterwarnings('ignore')

In [15]:
def list_unzip_files(folder_path, unzip_dbf=False):
    # List all files in the folder
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            relative_path = os.path.relpath(os.path.join(root, file), folder_path)
            all_files.append(relative_path)
    # If there are any dbf zip files, unzip them
    if unzip_dbf:
        for file_path in files:
            if file_path.endswith('.zip'):
                if not os.path.exists(f"{folder_path}/{file_path[:-4]}"):
                    with zipfile.ZipFile(f"{folder_path}/{file_path}", 'r') as zip_ref:
                        zip_ref.extractall(folder_path)
    return all_files

# Define the folder paths for shapefiles and csv files
folder_path = 'input_data/raw/fires'
files = list_unzip_files(folder_path, unzip_dbf=True)

In [16]:
def load_data(files, folder_path):
    dataframes = []
    # Load all shp and csv files
    for file_path in files:
        if file_path.endswith('.shp'):
            df = gpd.read_file(f"{folder_path}/{file_path}")
        elif file_path.endswith('.csv'):
            df = pd.read_csv(f"{folder_path}/{file_path}")
        else:
            continue
        df.columns = map(str.upper, df.columns)
        dataframes.append(df)
    # Concatenate all dataframes
    fire_data = pd.concat(dataframes, ignore_index=True)
    return fire_data

# Initialize an empty list to store dataframes
fire_data = load_data(files, folder_path)

In [17]:
def preprocess_fires(fire_data):
    # Drop duplicates time and location
    fire_data.drop_duplicates(subset=['ACQ_DATE', 'ACQ_TIME', 'LATITUDE', 'LONGITUDE'], inplace=True)
    # Drop unnecessary columns
    fire_data.drop(columns=['COUNTRY_ID', 'SCAN', 'TRACK', 'SATELLITE', 'CONFIDENCE', 'VERSION', 'FRP', 'DAYNIGHT', 'BRIGHTNESS', 'BRIGHT_T31', 'TYPE', 'GEOMETRY', 'INSTRUMENT'], inplace=True)
    # Drop rows with missing or nan values
    fire_data.dropna(inplace=True)
    # Transform the date to datetime format
    fire_data['ACQ_DATE'] = fire_data['ACQ_DATE'].apply(lambda x: pd.to_datetime(x).date())
    # Transform the time to integer format
    fire_data['ACQ_TIME'] = fire_data['ACQ_TIME'].astype(int)
    # Reduce the precision of the coordinates using two decimal places, which is approximately 1.1 km
    # (https://support.oxts.com/hc/en-us/articles/115002885125-Level-of-Resolution-of-Longitude-and-Latitude-Measurements)
    fire_data['LATITUDE'] = fire_data['LATITUDE'].round(2)
    fire_data['LONGITUDE'] = fire_data['LONGITUDE'].round(2)
    # Add an ID column and make it the first column
    fire_data['FIRE_ID'] = fire_data.index
    fire_data = fire_data[['FIRE_ID'] + [col for col in fire_data.columns if col != 'FIRE_ID']]
    return fire_data

# Preprocess the fire data
fire_data = preprocess_fires(fire_data)

In [18]:
def filter_fires_date(fire_data, start_date, end_date):
    # Filter the fire data based on the input parameters
    filtered_data = fire_data[(fire_data['ACQ_DATE'] >= start_date) & (fire_data['ACQ_DATE'] <= end_date)]
    # Reset the index
    filtered_data.reset_index(drop=True, inplace=True)
    return filtered_data

# Filter the fire data based on a specific date range
start_date = pd.to_datetime('2014-12-01').date() # Need the last month of 2014 for calculating rolling statistics
end_date = pd.to_datetime('today').date()
fire_data = filter_fires_date(fire_data, start_date, end_date)

In [19]:
def filter_data_border(data, path_to_border):
    # Load the shapefile containing the administrative borders of Ukraine
    ukraine_borders = gpd.read_file(path_to_border)
    # Ensure the data is a GeoDataFrame
    data_gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.LONGITUDE, data.LATITUDE))
    # Set the same coordinate reference system (CRS) for both GeoDataFrames
    data_gdf.set_crs(epsg=4326, inplace=True)
    ukraine_borders.set_crs(epsg=4326, inplace=True)
    # Perform a spatial join to filter datapoints within Ukraine borders
    data_in_ukraine = gpd.sjoin(data_gdf, ukraine_borders, how='inner')
    # Drop the geometry column as it's no longer needed
    data_in_ukraine.drop(columns=['geometry', 'source', 'name', 'index_right'], inplace=True)
    # Reset the index
    data_in_ukraine.reset_index(drop=True, inplace=True)
    # Make sure all columns are in uppercase
    data_in_ukraine.columns = map(str.upper, data_in_ukraine.columns)
    # Rename the ID column to OBLAST_ID
    data_in_ukraine.rename(columns={'ID': 'OBLAST_ID'}, inplace=True)
    # Generate a unique identifier for each grid cell
    data_in_ukraine['GRID_CELL'] = data_in_ukraine['LATITUDE'].astype(str) + '_' + data_in_ukraine['LONGITUDE'].astype(str)
    return data_in_ukraine

# Filter the fire data based on the administrative borders of Ukraine
fire_data = filter_data_border(fire_data, 'input_data/raw/ukr_borders/ua.shp') # https://simplemaps.com/gis/country/ua

In [20]:
def generate_counts(fire_data):
    # sort by date
    fire_data = fire_data.sort_values(by='ACQ_DATE')
    # Generate the day of the year
    fire_data['DAY_OF_YEAR'] = fire_data['ACQ_DATE'].apply(lambda x: x.timetuple().tm_yday)
    # Generate the number of fires per grid cell for the specific day (ACQ_DATE)
    fire_data['FIRE_COUNT_CELL'] = fire_data.groupby(['GRID_CELL', 'ACQ_DATE'])['ACQ_DATE'].transform('count')
    # Generate the number of fires per region (ID) for the specific day (ACQ_DATE)
    fire_data['FIRE_COUNT_OBLAST'] = fire_data.groupby(['OBLAST_ID', 'ACQ_DATE'])['ACQ_DATE'].transform('count')
    # Assign all rows with a value greater than 500 to 500 to avoid extreme values
    fire_data['FIRE_COUNT_OBLAST'] = fire_data['FIRE_COUNT_OBLAST'].apply(lambda x: 500 if x > 500 else x)
    # Generate average number of fires per grid cell, neighboring grid cells, and region for the last 7 and 30 days
    for i in [7, 30]:
        fire_data[f'FIRE_COUNT_CELL_AVG_{i}D'] = fire_data.groupby('GRID_CELL')['FIRE_COUNT_CELL'].transform(lambda x: x.rolling(i, min_periods=1).mean())
        fire_data[f'FIRE_COUNT_OBLAST_AVG_{i}D'] = fire_data.groupby('OBLAST_ID')['FIRE_COUNT_OBLAST'].transform(lambda x: x.rolling(i, min_periods=1).mean())
        # Assign all rows with a value greater than 500 to 500 to avoid extreme values
        fire_data[f'FIRE_COUNT_OBLAST_AVG_{i}D'] = fire_data[f'FIRE_COUNT_OBLAST_AVG_{i}D'].apply(lambda x: 500 if x > 500 else x)
    return fire_data

# Perform feature engineering
fire_data = generate_counts(fire_data)
# Filter the fire data based on a specific date range
start_date = pd.to_datetime('2015-01-01').date()
fire_data = filter_fires_date(fire_data, start_date, end_date)
# Save the preprocessed fire data
fire_data.to_csv('input_data/processed/fire_data.csv', index=False)
# Zip the preprocessed fire data
with zipfile.ZipFile('input_data/processed/fire_data.csv.zip', 'w') as zip_ref:
    zip_ref.write('input_data/processed/fire_data.csv', 'fire_data.csv')

In [21]:
def generate_population_density(path_to_population, round_precision=2):
    # Load the population density data
    pop_density_df = pd.read_csv(path_to_population)
    # Rename the columns to be more descriptive
    pop_density_df.columns = ['LONGITUDE', 'LATITUDE', 'POP_DENSITY']
    # Round the latitude and longitude to match the precision used in fire_data
    pop_density_df['LATITUDE'] = pop_density_df['LATITUDE'].round(round_precision)
    pop_density_df['LONGITUDE'] = pop_density_df['LONGITUDE'].round(round_precision)
    # Calculate the average population density for each grid cell
    pop_density_avg = pop_density_df.groupby(['LATITUDE', 'LONGITUDE'])['POP_DENSITY'].mean().reset_index()
    # Create a unique identifier for each grid cell
    pop_density_avg['GRID_CELL'] = pop_density_avg['LATITUDE'].astype(str) + '_' + pop_density_avg['LONGITUDE'].astype(str)
    # Rename the column to indicate it's the population density average per grid cell
    pop_density_avg.rename(columns={'POP_DENSITY': 'POP_DENSITY_CELL_AVG'}, inplace=True)
    # Assign all rows with a value greater than 250 to 250 to avoid extreme values
    pop_density_avg['POP_DENSITY_CELL_AVG'] = pop_density_avg['POP_DENSITY_CELL_AVG'].apply(lambda x: 250 if x > 250 else x)
    return pop_density_avg

# Calculate the population density for each grid cell
pop_density = generate_population_density('input_data/raw/population/ukr_pd_2020_1km_UNadj_ASCII_XYZ.csv')
# Save the population density data to a csv file
pop_density.to_csv('input_data/processed/pop_density.csv', index=False)

In [22]:
def merge_fire_pop_data(fire_data, pop_density):
    # Drop redundant columns
    pop_density.drop(columns=['LATITUDE', 'LONGITUDE'], inplace=True)
    # Merge the fire data with the population density data
    fire_data = fire_data.merge(pop_density, on='GRID_CELL', how='left')
    # Fill missing values with the average population density
    fire_data['POP_DENSITY_CELL_AVG'].fillna(fire_data['POP_DENSITY_CELL_AVG'].mean(), inplace=True)
    return fire_data

# Load the population density data
pop_density = pd.read_csv('input_data/processed/pop_density.csv')
# Load the preprocessed fire data
fire_data = pd.read_csv('input_data/processed/fire_data.csv')
# Merge the fire data with the population density data
fire_data = merge_fire_pop_data(fire_data, pop_density)

In [25]:
# Define the WGS84 projection (EPSG:4326)
wgs84_proj = Proj('epsg:4326')

def load_tif_with_reduced_resolution(tif_path, scale_factor):
    with rio.open(tif_path) as src:
        # Calculate the new shape
        new_height = int(src.height * scale_factor)
        new_width = int(src.width * scale_factor)
        # Read the data with resampling
        data = src.read(
            out_shape=(src.count, new_height, new_width),
            resampling=rio.enums.Resampling.bilinear
        )
        # Scale the transform
        transform_scaled = src.transform * src.transform.scale(
            (src.width / data.shape[-1]),
            (src.height / data.shape[-2])
        )
        # Get the UTM projection from the metadata
        utm_proj = Proj(src.crs)
        return data, np.array(transform_scaled), utm_proj

def transform_tif_coordinates(data, transform_tif, utm_proj):
    # transform the coordinates of the pixels
    lon = np.zeros(data.shape[1:])
    lat = np.zeros(data.shape[1:])
    for i in range(data.shape[1]):
        for j in range(data.shape[2]):
            lat[i][j], lon[i][j] = transform(utm_proj, wgs84_proj, transform_tif[2] + transform_tif[0] * j, transform_tif[5] + transform_tif[4] * i)
    return lon, lat

def convert_to_dataframe(data, lon, lat, map_dict):
    # Create a dataframe with the class of the pixel and its longitude and latitude value
    df = pd.DataFrame(data[0].flatten(), columns=['class'])
    # get longitude and latitude values and round them to 2 decimal places
    df['LONGITUDE'] = lon.flatten().round(2)
    df['LATITUDE'] = lat.flatten().round(2)
    # Delete all rows with class 0 (no data)
    df = df[df['class'] != 0]
    # Map the classes to the new classes
    df['CLASS'] = df['class'].map(map_dict)
    df.drop(columns=['class'], inplace=True)
    # Make sure all columns are in uppercase
    df.columns = map(str.upper, df.columns)
    return df

# Load the TIFF file with reduced resolution
# Reduce resolution by 95%, i.e., 10m resolution to 200m resolution
scale_factor = 0.05
# Dictionary to merge some of the classes into new classes
# Original classes: 0: 'No data', 1: 'Water', 2: 'Trees', 4: 'Flooded vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare ground', 9: 'Snow and ice', 10: 'Clouds', 11: 'Rangeland'
# New classes: 
# 0: ['No data', 'Water', 'Snow and ice', 'Clouds'], 
# 1: ['Trees'], 
# 2: ['Flooded vegetation', 'Bare ground', 'Rangeland'], 
# 3: ['Crops'], 
# 4: ['Built Area']
class_transform = {0: 0, 1: 0, 2: 1, 4: 2, 5: 3, 7: 4, 8: 2, 9: 0, 10: 0, 11: 2}

# Iterate over all tif files in the folder
for tif_file in list_unzip_files('input_data/raw/land_use')[3:]:
    if tif_file.endswith('.tif'):
        print(f"Initialized {tif_file}")
        # Load the TIFF file with reduced resolution
        data, transform_scaled, utm_proj = load_tif_with_reduced_resolution(f"input_data/raw/land_use/{tif_file}", scale_factor)
        # Transform the coordinates of the pixels
        lon, lat = transform_tif_coordinates(data, transform_scaled, utm_proj)
        # Convert the data to a dataframe
        land_use_df = convert_to_dataframe(data, lon, lat, class_transform)
        # Save dataframe as a csv file
        land_use_df.to_csv(f'input_data/processed/{tif_file[:-4]}_005.csv', index=False)
        print(f"Processed {tif_file}")

Initialized 36T_20220101-20230101.tif


In [23]:
def merge_dataframes(dataframes):
    # Merge the dataframes
    merged_df = pd.concat(dataframes, ignore_index=True)
    # Drop rows that are not in the Ukraine borders
    merged_df = filter_data_border(merged_df, 'input_data/raw/ukr_borders/ua.shp')
    # Reset the index
    merged_df.reset_index(drop=True, inplace=True)
    return merged_df

def class_percentage_cell(land_use_data):
    # One hot encode the class column in land_use_data
    land_use_percentage = pd.get_dummies(land_use_data, columns=['CLASS'], prefix='LAND_USE_CLASS', drop_first=True)
    # Drop the OBLAST_ID column as it's not needed
    land_use_percentage.drop(columns=['OBLAST_ID'], inplace=True)
    # Calculate the percentage of each class for each grid cell
    land_use_percentage = land_use_percentage.groupby(['GRID_CELL']).mean()
    land_use_percentage.reset_index(inplace=True)
    return land_use_percentage

# Load all csv files in the folder
dataframes = []
for csv_file in list_unzip_files('input_data/processed'):
    if csv_file.endswith('001.csv') and csv_file[:-8] in [x[:-4] for x in list_unzip_files('input_data/raw/land_use')]:
        dataframes.append(pd.read_csv(f"input_data/processed/{csv_file}"))
# Merge the dataframes
land_use_data = merge_dataframes(dataframes)
# Calculate the percentage of each class for each grid cell and neighbors
land_use_cells = class_percentage_cell(land_use_data)
# Save the dataframes to a csv file
land_use_cells.to_csv('input_data/processed/land_use_001.csv', index=False)
# Zip the land use data
with zipfile.ZipFile('input_data/processed/land_use_001.csv.zip', 'w') as zip_ref:
    zip_ref.write('input_data/processed/land_use_001.csv', 'land_use_001.csv')

In [24]:
# Load the land_use_percentage csv
land_use = pd.read_csv('input_data/processed/land_use_001.csv', index_col=0)
# Drop the redundant columns
land_use.drop(columns=['LATITUDE', 'LONGITUDE'], inplace=True)

# Merge the land_use_percentage with the fire_sample dataframe
aggregated_data = fire_data.merge(land_use, on='GRID_CELL', how='left')

# print the number of nan values in each column
print("NaN values before filling: {}".format(np.mean(aggregated_data.isnull().sum()/len(aggregated_data))))

# If there are any missing values in the columns for the classes, fill them with the most frequent value
for col in land_use.columns:
    aggregated_data[col] = aggregated_data[col].fillna(aggregated_data[col].mode()[0])

# print the number of nan values in each column
print("NaN values after filling: {}".format(np.mean(aggregated_data.isnull().sum()/len(aggregated_data))))

# Save the dataframes to a csv file
aggregated_data.to_csv('input_data/processed/aggregated_data_001.csv', index=False)
# Zip the aggregated data
with zipfile.ZipFile('input_data/processed/aggregated_data_001.csv.zip', 'w') as zip_ref:
    zip_ref.write('input_data/processed/aggregated_data_001.csv', 'aggregated_data_001.csv')

# Display the updated fire_sample dataframe
aggregated_data.head()

NaN values before filling: 0.052973832080296296
NaN values after filling: 0.0


,FIRE_ID,LATITUDE,LONGITUDE,ACQ_DATE,ACQ_TIME,OBLAST_ID,GRID_CELL,DAY_OF_YEAR,FIRE_COUNT_CELL,FIRE_COUNT_OBLAST,FIRE_COUNT_CELL_AVG_7D,FIRE_COUNT_OBLAST_AVG_7D,FIRE_COUNT_CELL_AVG_30D,FIRE_COUNT_OBLAST_AVG_30D,POP_DENSITY_CELL_AVG,LAND_USE_CLASS_1,LAND_USE_CLASS_2,LAND_USE_CLASS_3,LAND_USE_CLASS_4
0,921182,47.09,37.61,2015-01-01,930,UA14,47.09_37.61,1,3,4,1.857143,8.285714,3.400000,7.433333,250.000000,0.0,0.0,1.0,0.0
1,921184,47.09,37.61,2015-01-01,1106,UA14,47.09_37.61,1,3,4,2.000000,7.571429,3.380952,7.266667,250.000000,0.0,0.0,1.0,0.0
2,921183,47.09,37.61,2015-01-01,930,UA14,47.09_37.61,1,3,4,2.142857,6.857143,3.363636,7.100000,250.000000,0.0,0.0,1.0,0.0
3,921181,47.15,37.53,2015-01-01,930,UA14,47.15_37.53,1,1,4,1.000000,6.142857,1.000000,6.933333,250.000000,0.0,0.0,1.0,0.0
4,921185,50.51,28.74,2015-01-02,1054,UA18,50.51_28.74,2,1,1,1.000000,2.285714,1.000000,4.238095,16.663777,0.0,0.0,1.0,0.0
